In [4]:
"""Taken/adapted from https://github.com/tensorflow/models/tree/master/research/autoencoder"""

import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [19]:
#autoencoder class with deep fully connected architecture
class Autoencoder(object):

    def __init__(self, n_layers, transfer_function=tf.nn.softplus, optimizer=tf.train.AdamOptimizer()):
        self.n_layers = n_layers
        self.transfer = transfer_function

        network_weights = self._initialize_weights()
        self.weights = network_weights

        # model
        self.x = tf.placeholder(tf.float32, [None, self.n_layers[0]])
        self.hidden_encode = []
        h = self.x
        for layer in range(len(self.n_layers)-1):
            h = self.transfer(
                tf.add(tf.matmul(h, self.weights['encode'][layer]['w']),
                       self.weights['encode'][layer]['b']))
            self.hidden_encode.append(h)

        self.hidden_decode = []
        for layer in range(len(self.n_layers)-1):
            h = self.transfer(
                tf.add(tf.matmul(h, self.weights['decode'][layer]['w']),
                       self.weights['decode'][layer]['b']))
            self.hidden_decode.append(h)
        self.reconstruction = self.hidden_decode[-1]

        # cost
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)


    def _initialize_weights(self):
        all_weights = dict()
        initializer = tf.contrib.layers.xavier_initializer()
        # Encoding network weights
        encoder_weights = []
        for layer in range(len(self.n_layers)-1):
            w = tf.Variable(
                initializer((self.n_layers[layer], self.n_layers[layer + 1]),
                            dtype=tf.float32))
            b = tf.Variable(
                tf.zeros([self.n_layers[layer + 1]], dtype=tf.float32))
            encoder_weights.append({'w': w, 'b': b})
        # Decoding network weights
        decoder_weights = []
        for layer in range(len(self.n_layers)-1, 0, -1):
            w = tf.Variable(
                initializer((self.n_layers[layer], self.n_layers[layer - 1]),
                            dtype=tf.float32))
            b = tf.Variable(
                tf.zeros([self.n_layers[layer - 1]], dtype=tf.float32))
            decoder_weights.append({'w': w, 'b': b})
        all_weights['encode'] = encoder_weights
        all_weights['decode'] = decoder_weights
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x: X})
        return cost

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict={self.x: X})

    def transform(self, X):
        return self.sess.run(self.hidden_encode[-1], feed_dict={self.x: X})

    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['encode'][-1]['b'])
        return self.sess.run(self.reconstruction, feed_dict={self.hidden_encode[-1]: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict={self.x: X})

    def getWeights(self):
        raise NotImplementedError
        return self.sess.run(self.weights)

    def getBiases(self):
        raise NotImplementedError
        return self.sess.run(self.weights)

In [20]:
#def main():
# Import data
mnist = read_data_sets("MNIST_data", one_hot=False)
n = len(mnist.train.images)
epochs = 1
batch_size = 50
iterations = int(n / batch_size * epochs)
autoencoder = Autoencoder(n_layers=[784, 200],
                          transfer_function = tf.nn.softplus,
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001))
for i in range(iterations):
    batch = mnist.train.next_batch(batch_size)
    cost = autoencoder.partial_fit(batch[0])
    if i % 500 == 0:
        print "cost for this batch: " + str(cost)
        print("cost of test batch: " + str(autoencoder.calc_total_cost(mnist.test.next_batch(batch_size)[0])))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
[{'b': <tf.Variable 'Variable_17:0' shape=(200,) dtype=float32_ref>, 'w': <tf.Variable 'Variable_16:0' shape=(784, 200) dtype=float32_ref>}]
[{'b': <tf.Variable 'Variable_19:0' shape=(784,) dtype=float32_ref>, 'w': <tf.Variable 'Variable_18:0' shape=(200, 784) dtype=float32_ref>}]
0
cost for this batch: 9456.501
cost of test batch: 7701.3506
cost for this batch: 339.09537
cost of test batch: 308.5683
cost for this batch: 253.99118
cost of test batch: 237.88475


55000